In [4]:
import cv2
import mediapipe as mp
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [8]:
import csv
import cv2
import mediapipe as mp
import math
import os

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = tuple(a)
    b = tuple(b)
    c = tuple(c)

    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = abs(math.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Input folder containing video files
input_folder = 'A:\gym_tracker-ai\gym_tracker-ai\Diagonal Left and Right Dumbell Lateral Raises test #1'  # Replace with your folder containing video files
output_csv = 'Lateral_Raises_distances_and_frontal_angles.csv'

# Open the CSV file to write distances and angles
with open(output_csv, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Define relevant distances (pairs of landmarks) and angles to calculate
    relevant_distances = [
        (11, 13),  # Left shoulder -> Left elbow
        (12, 14),  # Right shoulder -> Right elbow
        (23, 11),  # Left hip -> Left shoulder
        (24, 12),  # Right hip -> Right shoulder
        (25, 23),  # Left hip -> Left knee
        (26, 24),  # Right hip -> Right knee
        (27, 25),  # Left knee -> Left ankle
        (28, 26),  # Right knee -> Right ankle
        (11, 12),  # Left shoulder -> Right shoulder (for chest/shoulder width)
        (13, 14),  # Left elbow -> Right elbow (for arm width)
    ]

    # Write header row for CSV (relevant distances + angles)
    header = ['Sequence']
    for d in relevant_distances:
        header.append(f'distance_{d[0]}_{d[1]}')
    for name, _, _, _ in [
        ("left_elbow", 11, 13, 15),
        ("right_elbow", 12, 14, 16),
        ("left_shoulder", 23, 11, 13),
        ("right_shoulder", 24, 12, 14),
        ("left_hip", 25, 23, 11),
        ("right_hip", 26, 24, 12),
        ("left_knee", 23, 25, 27),
        ("right_knee", 24, 26, 28),
    ]:
        header.append(f'{name}_angle')
    csv_writer.writerow(header)

    # Process each video in the folder
    sequence_id = 0  # Start sequence id
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)
        if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        print(f"Processing video: {video_file}")

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert the image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Process the image
                results = pose.process(image)

                # Convert back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract landmarks and compute distances and angles
                try:
                    if results.pose_landmarks:
                        landmarks = results.pose_landmarks.landmark

                        # Calculate relevant distances
                        distances = []
                        for i, j in relevant_distances:
                            dx = landmarks[j].x - landmarks[i].x
                            dy = landmarks[j].y - landmarks[i].y
                            distance = math.sqrt(dx ** 2 + dy ** 2)
                            distances.append(distance)

                        # Calculate angles in the frontal plane
                        angles = []
                        for name, idx_a, idx_b, idx_c in [
                            ("left_elbow", 11, 13, 15),
                            ("right_elbow", 12, 14, 16),
                            ("left_shoulder", 23, 11, 13),
                            ("right_shoulder", 24, 12, 14),
                            ("left_hip", 25, 23, 11),
                            ("right_hip", 26, 24, 12),
                            ("left_knee", 23, 25, 27),
                            ("right_knee", 24, 26, 28),
                        ]:
                            a = [landmarks[idx_a].x, landmarks[idx_a].y]
                            b = [landmarks[idx_b].x, landmarks[idx_b].y]
                            c = [landmarks[idx_c].x, landmarks[idx_c].y]
                            angle = calculate_angle(a, b, c)
                            angles.append(angle)

                        # Write sequence ID, distances, and angles to CSV
                        csv_writer.writerow([sequence_id] + distances + angles)

                except Exception as e:
                    print(f"Error processing frame: {e}")

                # Render detections
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )

                # Show the frame (optional)
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()

        # Increment sequence ID for the next video
        sequence_id += 1

    cv2.destroyAllWindows()

print(f"Relevant distances and angles with sequence IDs saved to {output_csv}")


Processing video: 02.mp4


AttributeError: module 'google.protobuf.message_factory' has no attribute 'GetMessageClass'

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [37]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a fixed sequence length (e.g., 20 frames per sequence)
max_sequence_length = 20

# Prepare your sequences
sequence_data = []

for sequence_id in df['Sequence'].unique():
    # Filter rows that belong to the same rep (sequence)
    sequence_frames = df[df['Sequence'] == sequence_id]
    
    # Get features (drop 'Class' and 'Sequence' columns)
    features = sequence_frames.drop(columns=['Class', 'Sequence']).values
    
    # Pad or trim the sequence to a fixed length
    padded_sequence = pad_sequences([features], maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')
    
    # Get the label for this sequence (assuming it's the same for all frames in a sequence)
    label = sequence_frames['Class'].iloc[0]
    
    # Append the padded sequence and its label
    sequence_data.append((padded_sequence[0], label))

# Convert to NumPy arrays
X = np.array([item[0] for item in sequence_data])  # Shape (num_sequences, num_frames, num_features)
y = np.array([item[1] for item in sequence_data])  # Shape (num_sequences,)

# Check the shape of your data
print(X.shape, y.shape)


(21, 20, 18) (21,)


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential()

# Add LSTM layer with the input shape (20 frames, 18 features)
model.add(LSTM(64, input_shape=(20, 18), return_sequences=False))  # 'return_sequences=False' for a final classification output

# Optional: Add dropout for regularization
model.add(Dropout(0.2))

# Add Dense layer to output the class prediction (binary classification: 0 or 1)
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model to inspect the architecture
model.summary()

# Train the model
history = model.fit(X, y, epochs=10, batch_size=4, validation_split=0.2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                21248     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 21,313
Trainable params: 21,313
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
4/4 [==============================] - 2s 170ms/step - loss: 0.9642 - accuracy: 0.1875 - val_loss: 0.9267 - val_accuracy: 0.2000
Epoch 2/10
4/4 [==============================] - 0s 16ms/step - loss: 0.9347 - accuracy: 0.1250 - val_loss: 0.7909 - val_accuracy: 0.2000
Epoch 3/10
4/4 [===================

In [39]:
# Evaluate the model on the same data or separate test data
loss, accuracy = model.evaluate(X, y)
print(f"Model Loss: {loss}")
print(f"Model Accuracy: {accuracy}")


1/1 [==============================] - 0s 20ms/step - loss: 0.3599 - accuracy: 0.9524
Model Loss: 0.3598628342151642
Model Accuracy: 0.9523809552192688


In [40]:
import cv2
import numpy as np
import mediapipe as mp
import math
from tensorflow.keras.models import load_model

# Load the trained LSTM model (ensure you've saved your model after training)
# model = load_model('path_to_your_trained_model.h5')  # Uncomment if needed
# For now, we assume the model is already available

# Initialize Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Initialize webcam
cap = cv2.VideoCapture(1)  # Webcam ID 1

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = abs(math.degrees(radians))
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Define the angle mapping for the frontal plane
frontal_plane_angles = [
    ("left_elbow", 11, 13, 15),
    ("right_elbow", 12, 14, 16),
    ("left_shoulder", 23, 11, 13),
    ("right_shoulder", 24, 12, 14),
    ("left_hip", 25, 23, 11),
    ("right_hip", 26, 24, 12),
    ("left_knee", 23, 25, 27),
    ("right_knee", 24, 26, 28),
]

# Function to calculate distances between selected landmarks
def calculate_distances(landmarks):
    relevant_pairs = [
        (11, 13),  # Left shoulder -> Left elbow
        (12, 14),  # Right shoulder -> Right elbow
        (23, 11),  # Left hip -> Left shoulder
        (24, 12),  # Right hip -> Right shoulder
        (25, 23),  # Left hip -> Left knee
        (26, 24),  # Right hip -> Right knee
        (27, 25),  # Left knee -> Left ankle
        (28, 26),  # Right knee -> Right ankle
        (11, 12),  # Left shoulder -> Right shoulder
        (13, 14),  # Left elbow -> Right elbow
    ]

    distances = []
    for i, j in relevant_pairs:
        dx = landmarks[j].x - landmarks[i].x
        dy = landmarks[j].y - landmarks[i].y
        distance = math.sqrt(dx ** 2 + dy ** 2)
        distances.append(distance)
    
    return distances

# Prepare for sequence collection
sequence_length = 20  # Use 20 frames for each sequence
sequence = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the frame
        results = pose.process(image)

        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks and calculate distances and angles
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Calculate distances
                distances = calculate_distances(landmarks)

                # Calculate angles in the frontal plane
                angles = []
                for name, idx_a, idx_b, idx_c in frontal_plane_angles:
                    a = [landmarks[idx_a].x, landmarks[idx_a].y]
                    b = [landmarks[idx_b].x, landmarks[idx_b].y]
                    c = [landmarks[idx_c].x, landmarks[idx_c].y]
                    angle = calculate_angle(a, b, c)
                    angles.append(angle)

                # Combine distances and angles as features for prediction
                features = distances + angles
                sequence.append(features)

                # Ensure the sequence is of the correct length
                if len(sequence) > sequence_length:
                    sequence.pop(0)  # Remove the first frame to keep the sequence length fixed

                # If the sequence is full, predict the class
                if len(sequence) == sequence_length:
                    # Convert the sequence to a NumPy array and reshape for LSTM input
                    sequence_data = np.array(sequence).reshape(1, sequence_length, len(features))

                    # Make prediction
                    predicted_class = model.predict(sequence_data)
                    predicted_class = (predicted_class > 0.5).astype(int)  # Convert to 0 or 1

                    # Display the predicted class on the frame
                    cv2.putText(
                        image,
                        f'Predicted Class: {predicted_class[0][0]}',
                        (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA
                    )

        except Exception as e:
            print(f"Error processing frame: {e}")

        # Render landmarks
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # Show the frame with the predicted class
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 26ms/step


In [9]:
import csv
import cv2
import mediapipe as mp  
import math
import os

# Initialize Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Input folder containing video files
input_folder = 'A:\gym_tracker-ai\gym_tracker-ai\Diagonal Left and Right Squats Hands Front Raised Test #1'
output_csv = 'Diagonal Left and Right Squats Hands Front Raised.csv'

# Open the CSV file to write the angles
with open(output_csv, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Write header row
    header = [f'Angle_{i}' for i in [11, 12, 13, 14, 23, 24, 25, 26]]
    csv_writer.writerow(header)

    # Process each video in the folder
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)
        if not video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        print(f"Processing video: {video_file}")

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Recolor image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make detection
                results = pose.process(image)

                # Recolor back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract landmarks and calculate angles
                try:
                    if results.pose_landmarks:
                        landmarks = results.pose_landmarks.landmark
                        angles = []

                        # Extract required points for angle calculation
                        def get_coords(idx):
                            return [landmarks[idx].x, landmarks[idx].y]

                        # Define angle mapping
                        angle_map = {
                            11: (23, 11, 13),
                            12: (24, 12, 14),
                            13: (11, 13, 15),
                            14: (12, 14, 16),
                            23: (25, 23, 24),
                            24: (26, 24, 23),
                            25: (27, 25, 23),
                            26: (28, 26, 24)
                        }

                        for i, (a, b, c) in angle_map.items():
                            angle = calculate_angle(get_coords(a), get_coords(b), get_coords(c))
                            angles.append(angle)

                        # Write angles to CSV
                        csv_writer.writerow([video_file] + angles)

                except Exception as e:
                    print(f"Error processing frame: {e}")

                # Render detections
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )

                # Show the frame (optional, can be commented out for faster processing)
                cv2.imshow('Mediapipe Feed', image)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()

    cv2.destroyAllWindows()

print(f"Pose angles saved to {output_csv}")


Processing video: Nested Sequence 01.mp4


AttributeError: module 'google.protobuf.message_factory' has no attribute 'GetMessageClass'

In [5]:
results.pose_landmarks

NameError: name 'results' is not defined